In [1]:
!nvidia-smi

Thu Mar 21 13:19:16 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.147.05   Driver Version: 525.147.05   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:01:00.0 Off |                  Off |
|  0%   49C    P8    19W / 450W |      0MiB / 24564MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [1]:
# !pip install display_tree
# !pip3 install requests==2.27.0
# !pip show requests

In [1]:
import os

os.environ["http_proxy"]  = "http://proxy.ad.speechpro.com:3128"
os.environ["https_proxy"] = "http://proxy.ad.speechpro.com:3128"
os.environ["ftp_proxy"]   = "http://proxy.ad.speechpro.com:3128"

os.environ['CURL_CA_BUNDLE'] = ''

In [9]:
from cutils import create_chunk_dataset, del_folder, load_checkpoint
from calc_content import create_hubert_content
from clustering import incremental_clustering, PseudoPhonemes
from labling import Text2PseudoPhonemes
from models import TextEncoder

from pathlib import Path
from typing import *
import json

import wget
import torch
from safetensors.torch import load_file
from torch.utils.data import DataLoader

def path(_path):
    return "./examples/" / Path(_path)

In [2]:
create_chunk_dataset("../../NIR/RuDevices", out_dataset=path("rudevices_chunk"), display=True)

rudevices_chunk/
├── 3/
│   ├── 9/
│   │   ├── 0f1576b3-073d-4d08-b6c1-98b0400cad8e.txt
│   │   ├── 0f1576b3-073d-4d08-b6c1-98b0400cad8e.wav
│   │   ├── 4750d7d6-cf38-440f-9449-19e5a500acf7.txt
│   │   ├── 4750d7d6-cf38-440f-9449-19e5a500acf7.wav
│   │   ├── 49d0f2e9-f9ae-4186-8ed9-e6330f06af58.txt
│   │   ├── 49d0f2e9-f9ae-4186-8ed9-e6330f06af58.wav
│   │   ├── eeb16c30-6100-430b-b871-c02c6bd740ee.txt
│   │   └── eeb16c30-6100-430b-b871-c02c6bd740ee.wav
│   └── d/
│       ├── 0f9afddb-c797-44d1-b9e7-2d21bd6b9780.txt
│       ├── 0f9afddb-c797-44d1-b9e7-2d21bd6b9780.wav
│       ├── 7fec0bb5-2e94-43dd-bd69-6b5700a56563.txt
│       ├── 7fec0bb5-2e94-43dd-bd69-6b5700a56563.wav
│       ├── c51d1a43-7f92-4879-bcf0-2766bb7d2287.txt
│       ├── c51d1a43-7f92-4879-bcf0-2766bb7d2287.wav
│       ├── da2f8a2a-4b81-4e83-9370-89a87ec02ff7.txt
│       └── da2f8a2a-4b81-4e83-9370-89a87ec02ff7.wav
└── d/
    ├── 5/
    │   ├── 2966daa6-6cd8-4444-bc2b-5eacc7b3912f.txt
    │   ├── 2966daa6-6cd8-4444-bc2b

In [6]:
!ls examples/rudevices_chunk/1/1/

314cda50-5039-44ce-bd93-9cc9dc66daf2.txt
314cda50-5039-44ce-bd93-9cc9dc66daf2.wav
809f7596-88dd-4e5a-95f4-7b8c322ee5ea.txt
809f7596-88dd-4e5a-95f4-7b8c322ee5ea.wav
b56f9ded-9365-416d-be97-f9c77746d163.txt
b56f9ded-9365-416d-be97-f9c77746d163.wav
d18f257d-ca75-4b8d-89a3-18c343b83990.txt
d18f257d-ca75-4b8d-89a3-18c343b83990.wav


In [ ]:
# git clone lengyue233/content-vec-best

In [4]:
# !mkdir ../../hfmodels
# download_hf_model("lengyue233/content-vec-best", "../../hfmodels") #TODO: Add final proj layer in lengyue233/content-vec-best

In [5]:
#TODO: Add final proj layer in lengyue233/content-vec-best
del_folder(path("extracted_contents"))
create_hubert_content(
    data_dir=path("rudevices_chunk"), out_dir=path("extracted_contents"), 
    njobs=1, pretrain_path="../../hfmodels/content-vec-best",
    )

n_jobs=1
{'hmodel_id': '../../hfmodels/content-vec-best', 'data_sr': 16000, 'out_dir': PosixPath('examples/extracted_contents'), 'device': 'cuda', 'rel_to': PosixPath('examples/rudevices_chunk')}


100%|██████████| 16/16 [00:05<00:00,  2.93it/s]


In [5]:
incremental_clustering(
    path("extracted_contents"), path("clusters/clusters.pt"),
    n_clusters=100, batch_size=5, data_pattern="*.content.pt"
    )

hubert contents shape: (874, 768), 2.56 MB
hubert contents shape: (670, 768), 1.96 MB
hubert contents shape: (663, 768), 1.94 MB
hubert contents shape: (392, 768), 1.15 MB
Clustering time 0.36 seconds


In [6]:
cluster = PseudoPhonemes(path("clusters/clusters.pt"))
cluster.build_clusters()

In [7]:
data = torch.load(path("clusters/clusters.pt"))
print(data['cluster_centers'].shape)
item = (data['cluster_centers'][50] + data['cluster_centers'][25]).reshape(1, -1)
print(cluster.predict_cluster_center(item))
print(cluster.get_cluster_center(item).shape)

(100, 768)
[50]
(1, 768)


In [7]:
!ls ../../hfmodels/content-vec-best

config.json  convert.py  pytorch_model.bin  README.md


In [3]:
conf_p = "../../NIR/YourTTS-RU-RUSLAN-April-30-2023_03+48PM-0000000"
dataset = Text2PseudoPhonemes(
    path("rudevices_chunk"), path("extracted_contents"), path("clusters/clusters.pt"), 
    "../../hfmodels/content-vec-best", None, conf_p+"/config.json",
    )

[20:09:51] /mnt/storage/kocharyan/sambashare/ITMO_LABS/ML/venv/local/lib/python3.8/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()



In [4]:
next(iter(DataLoader(dataset, batch_size=1, collate_fn=lambda batch: batch)))

[{'tokens': tensor([142, 143, 135, 129, 132, 145, 175, 140, 127, 136, 145, 135, 175, 144,
          132, 143, 135, 127, 138, 175, 129, 141, 175, 129, 144, 132, 175, 145,
          158, 133, 137, 135, 132]),
  'pseudo_ph': tensor([14, 14, 91, 91, 13, 13, 13, 13, 13, 99, 99, 92, 95, 95, 51, 77, 77, 77,
          77, 57, 57, 57, 57, 57, 50, 50, 50, 27, 14, 33, 33, 33, 63, 63, 10, 93,
          93, 93, 13, 13, 13, 13, 99, 81, 81, 69, 31, 63, 65, 51, 51, 51, 51, 51,
          99, 87, 92, 44, 44, 56, 56, 13, 13, 78, 91, 91, 91, 77, 77, 77, 35, 35,
          35, 35, 35, 49, 78, 78, 58, 53,  0, 55, 49,  1, 90, 92, 44, 44, 56, 56,
          51, 99, 99, 69, 92, 44, 96,  9,  9, 35, 35, 35, 35, 57, 57, 90, 38, 19,
          32, 32, 32, 31, 92, 44, 96, 51, 51, 51, 51, 25, 25, 14,  8, 48, 14, 48,
          16, 14, 62, 62, 14, 84, 84, 84, 14, 84, 11, 11, 14, 14]),
  'text': 'привет найти сериал во все тяжкие'}]

In [7]:
_ = next(iter(DataLoader(dataset, batch_size=1, collate_fn=lambda batch: batch)))[0]
print(len(_['tokens']), len(_['pseudo_ph']), len(_['text']))

33 140 33


In [41]:
# !mkdir ckpts
!cp ../../NIR/YourTTS-RU-RUSLAN-April-30-2023_03+48PM-0000000/best_model_158996.pth ckpts/yourtts_ruslan.pth

In [43]:
# or https://huggingface.co/joefox/tts_vits_ru_hf

with open("../../NIR/YourTTS-RU-RUSLAN-April-30-2023_03+48PM-0000000/config.json", 'r') as f:
    data = json.load(f)
data['characters']

{'characters_class': 'TTS.tts.models.vits.VitsCharacters',
 'vocab_dict': None,
 'pad': '_',
 'eos': '&',
 'bos': '*',
 'blank': None,
 'characters': "ЁАБВГДЕЖЗИЙКЛМНОПСТУФХЦЧШЩЪЫЬЭЮЯабвгдежзийклмнопстуфхцчшщъыьэюяё¯·ßàáâãäæçèéêëìíîïñòóôõöùúûüÿāąćēęěīıłńōőœśūűźżǎǐǒǔабвгдежзийклмнопрстуфхцчшщъыьэюяёєіїґ–!'(),-.:;? ",
 'punctuations': "!'(),-.:;? ",
 'phonemes': '',
 'is_unique': True,
 'is_sorted': True}

In [5]:
# symbols_dict = {
#     "_pad" : '_',
#     "_punctuation" : ' !+,-.:;?«»—',
#     "_letters" : 'абвгдежзийклмнопрстуфхцчшщъыьэюяё',
#     "_letters_ipa" : "ɑɐɒæɓʙβɔɕçɗɖðʤəɘɚɛɜɝɞɟʄɡɠɢʛɦɧħɥʜɨɪʝɭɬɫɮʟɱɯɰŋɳɲɴøɵɸθœɶʘɹɺɾɻʀʁɽʂʃʈʧʉʊʋⱱʌɣɤʍχʎʏʑʐʒʔʡʕʢǀǁǂǃˈˌːˑʼʴʰʱʲʷˠˤ˞↓↑→↗↘'̩'ᵻ",
# }

# len(dataset.symbols) #dataset symbols len

# #OR

# dataset._symbol_to_id = { " ": 0, "!": 37, "+": 40, ",": 39, ".": 36, ":": 2, "<pad>": 42, "<unk>": 41, "?": 38, "_": 1, "а": 3, "б": 4, "в": 5, "г": 6, "д": 7, "е": 8, "ж": 10, "з": 11, "и": 12, "й": 13, "к": 14, "л": 15, "м": 16, "н": 17, "о": 18, "п": 19, "р": 20, "с": 21, "т": 22, "у": 23, "ф": 24, "х": 25, "ц": 26, "ч": 27, "ш": 28, "щ": 29, "ъ": 30, "ы": 31, "ь": 32, "э": 33, "ю": 34, "я": 35, "ё": 9 }

In [7]:
len(dataset.symbols), len(dataset._symbol_to_id), len({s: i for i, s in enumerate(dataset.symbols)}) #WTF!

(177, 134, 134)

In [8]:
n_vocab         = len(dataset.symbols) # 177
inter_channels  = 192
hidden_channels = 192
filter_channels = 768
n_heads         = 2
n_layers        = 6 # 10 for YourTTS
kernel_size     = 3
p_dropout       = .1

prior_encoder = TextEncoder(
    n_vocab, inter_channels, hidden_channels, filter_channels, 
    n_heads, n_layers, kernel_size, p_dropout,
)   

In [10]:
prior_encoder = load_checkpoint(prior_encoder, 
                                "ckpts/yourtts_ruslan.pth", 
                                "text_encoder", False)